Includes data cleaning, feature engineering, dataset merging and train/test ready pickle outputs

In [ ]:
from sklearn import svm
import pandas as pd
import numpy as np
import tldextract as tld
from dateutil.relativedelta import relativedelta
from sklearn import datasets
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import GridSearchCV
from IPython.display import display, HTML
from tqdm import *
import matplotlib.pyplot as plt
from datetime import datetime, date
from collections import Counter 

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
%matplotlib inline

In [ ]:
df_reduced_train =  pd.read_csv("./data/reduced_train.csv")
#df_user_properties = pd.read_csv("./data/user_properties.csv")
#df_purchases =  pd.read_csv("./data/purchases.csv")
#df_pw_train =  pd.read_csv("./data/pw_train.csv")
#df_pw_test =  pd.read_csv("./data/pw_test.csv")
df_reduced_test =  pd.read_csv("./data/reduced_test.csv")
#df_search_terms_desktop =  pd.read_csv("./data/search_terms_desktop.csv")

In [ ]:
!free -h

In [ ]:
def count_list(field):
    number_items = str(field).count(";")+1
    return number_items

def create_feat_eng(df):
    df["all_domain"] = df["all_domain"].astype(str)
    df["total_domains"] = df.apply(lambda x: count_list(x.all_domain), axis=1)
    df["total_categories"] = df.apply(lambda x: count_list(x.all_category), axis=1)

    df['day_start'] = df.start.map(lambda x: datetime(int(x.split("-")[0]), int(x.split("-")[1]), int(x.split("-")[2].split("T")[0])).weekday())
    df['day_end'] = df.end.map(lambda x: datetime(int(x.split("-")[0]), int(x.split("-")[1]), int(x.split("-")[2].split("T")[0])).weekday())
    df['time_start'] = df.start.map(lambda x: int(x.split("-")[2].split("T")[1].split("Z")[0].split(":")[0]))
    df['time_end'] = df.end.map(lambda x: int(x.split("-")[2].split("T")[1].split("Z")[0].split(":")[0]))

    df['start_till_weekend'] = df.day_start.map(lambda x: 6 - x)
    df['end_till_weekend'] = df.day_end.map(lambda x: 6 - x)

    df['date_start'] = df.start.map(lambda x: (int(x.split("-")[2].split("T")[0])))
    df['date_end'] = df.end.map(lambda x: (int(x.split("-")[2].split("T")[0])))
    df["startend"] = df["start"] + df["end"]
    df['duration'] = df.startend.map(lambda x: ((datetime(int(x.split("Z")[1].split("-")[0]),  int(x.split("Z")[1].split("-")[1]), int(x.split("Z")[1].split("-")[2].split("T")[0]), int(x.split("Z")[1].split("-")[2].split("T")[1].split(":")[0]), int(x.split("Z")[1].split("-")[2].split("T")[1].split(":")[1]), int(x.split("Z")[1].split("-")[2].split("T")[1].split(":")[2]))) - (datetime(int(x.split("Z")[0].split("-")[0]),  int(x.split("Z")[0].split("-")[1]), int(x.split("Z")[0].split("-")[2].split("T")[0]), int(x.split("Z")[0].split("-")[2].split("T")[1].split(":")[0]), int(x.split("Z")[0].split("-")[2].split("T")[1].split(":")[1]), int(x.split("Z")[0].split("-")[2].split("T")[1].split(":")[2])))).total_seconds() )
    return df

df_reduced_train = create_feat_eng(df_reduced_train)
df_reduced_train = create_feat_eng(df_reduced_test)

In [ ]:
# category list parsing from train set
cat_col_tr = []
for cat in df_reduced_train['all_category'].values:
    tmp = []
    try:
        for link in cat.split(';'):
            tmp.append(link)
        cat_col_tr.append(tmp)
    except:
        cat_col_tr.append([])
       

cat_col_te = []
for cat in df_reduced_test['all_category'].values:
    tmp = []
    try:
        for link in cat.split(';'):
            tmp.append(link)
        cat_col_te.append(tmp)
    except:
        cat_col_te.append([])

# test category filling 
# categs = set(list(set([x for y in cat_col_te for x in y if x not in ['']])) + list(set([x for y in cat_col_tr for x in y if x not in ['']]))) 
# catDF = pd.DataFrame(columns=categs, index=xtmp.session_id.values)
# catDF.fillna(0, inplace=True)
# catDF = catDF.reset_index()
# catDF.rename(columns = {'index':'session_id'}, inplace=True)

# for i in tqdm(range(len(cat_col_te))):
#     try:
#         catDF.loc[i, cat_col_te[i]] = 1
#     except:
#         continue
        
# train category filling 
categs = set(list(set([x for y in cat_col_te for x in y if x not in ['']])) + list(set([x for y in cat_col_tr for x in y if x not in ['']]))) 
catDF = pd.DataFrame(columns=categs, index=df_reduced_train.session_id.values)
catDF.fillna(0, inplace=True)
catDF = catDF.reset_index()
catDF.rename(columns = {'index':'session_id'}, inplace=True)

for i in tqdm(range(len(cat_col_tr))):
    try:
        catDF.loc[i, cat_col_tr[i]] = 1
    except:
        continue

In [ ]:
# mobile app list parsing from train set
app_col_tr = []
for app in df_reduced_train['all_app'].values:
    tmp = []
    try:
        for link in app.split(';'):
            tmp.append(link)
        app_col_tr.append(tmp)
    except:
        app_col_tr.append([])

# mobile app list parsing from test set
app_col_te = []
for app in df_reduced_test['all_app'].values:
    tmp = []
    try:
        for link in app.split(';'):
            tmp.append(link)
        app_col_te.append(tmp)
    except:
        app_col_te.append([])
        
a =  list([x for y in app_col_te for x in y])

letter_counts = Counter(a)
dfa = pd.DataFrame(list(letter_counts.items()), columns=['app', 'count'])
dfa = dfa[dfa.app != ""]
dfa = dfa.sort_values('count', ascending=0)
dfa = dfa[dfa['count'] > 0]
dfa = dfa.app[:300]

# test
domDF1 = pd.DataFrame(columns=dfa.values, index=df_reduced_train.session_id.values)
domDF1.fillna(0, inplace=True)
domDF1 = domDF1.reset_index()

domDF1.rename(columns = {'index':'session_id'}, inplace=True)

for i in tqdm(range(len(app_col_tr))):
    try:
        domDF1.loc[i,app_col_tr[i]] = 1
    except:
        continue
        
# test
domDF = pd.DataFrame(columns=dfa.values, index=df_reduced_test.session_id.values)
domDF.fillna(0, inplace=True)
domDF = domDF.reset_index()

domDF.rename(columns = {'index':'session_id'}, inplace=True)

for i in tqdm(range(len(app_col_te))):
    try:
        domDF.loc[i,app_col_te[i]] = 1
    except:
        continue

In [ ]:
df_reduced_train = pd.merge(df_reduced_train, catDF, on="session_id")

In [ ]:
# generate top domains. load next block instead if data is ready
def get_domain(x):
    return tld.extract(x)[1]

domain_col_te = []
for domains in df_reduced_test['all_domain'].values:
    tmp = []
    try:
        for link in domains.split(';'):
            tmp.append(get_domain(link))
        domain_col_te.append(tmp)
    except:
        domain_col_te.append([])
        
a =  list([x for y in domain_col_te for x in y])
letter_counts = Counter(a)
dfa = pd.DataFrame(list(letter_counts.items()), columns=['domain', 'count'])
dfa = dfa[dfa.domain != ""]
dfa = dfa.sort_values('count', ascending=0)
dfa = dfa[dfa['count'] > 10]
top = dfa['domain'].values.ravel()

In [ ]:

top = pd.read_csv("./top500.csv", header=None)
top = list(top.values.ravel())

In [ ]:
def calculate_durations(picklepath_durations):
    durations = pd.read_pickle(picklepath_durations)
    # keep only domain durations from the top domain list
    durations = durations[durations['domain'].isin(top)]
    # chunking due to sparsity, avoid mem issues
    res = pd.DataFrame()
    for end_chunk in range(100000,durations.shape[0]+1,100000):
        print(end_chunk)
        res1 = durations.iloc[end_chunk-100000:end_chunk,:].reset_index(drop=False)\
            .pivot_table(values='domain_duration', index=['index', 'session_id'], columns='domain')
        try:
            res2 = res1.reset_index(drop=False).set_index("session_id").groupby(level=0).sum().drop("index", axis=1)
            display(res2.head())
        except:
            # bypass pandas bug
            display(res1.head())
            res1.index = res1.index.droplevel(0)
            res2 = res1.groupby(level=0).sum().drop("index", axis=1)  
        res = res.append(res2)
    res = res.groupby(level=0).sum()
    res = res.reset_index().rename(columns={"index": "session_id"})
    res["session_id"] = res["session_id"].astype(int)
    return res

In [ ]:
res = calculate_durations("./data/durations.pkl")
df = pd.merge(df_reduced_train, res, on="session_id", how="outer")

In [ ]:
df.fillna(0, inplace=True)
df = pd.get_dummies(df, columns=["day_start", "day_end", "device_session"])
y = df["geslacht"].factorize()[0]

In [ ]:
df.drop(["panelist_id","all_domain", "all_app", "all_category", "start", "end",
         "startend", "session_id", "geslacht"], axis=1, inplace=True)
df.head(2)

In [ ]:
df.shape

In [ ]:
pd.to_pickle(df, "X_train_.pkl")
pd.to_pickle(y, "y_train_.pkl")

**test set prep**

In [ ]:
# generate test categories
cat_col_tr = []
for cat in df_reduced_train['all_category'].values:
    tmp = []
    try:
        for link in cat.split(';'):
            tmp.append(link)
        cat_col_tr.append(tmp)
    except:
        cat_col_tr.append([])
       

cat_col_te = []
for cat in df_reduced_test['all_category'].values:
    tmp = []
    try:
        for link in cat.split(';'):
            tmp.append(link)
        cat_col_te.append(tmp)
    except:
        cat_col_te.append([])

# test
categs = set(list(set([x for y in cat_col_te for x in y if x not in ['']])) + list(set([x for y in cat_col_tr for x in y if x not in ['']]))) 
catDF = pd.DataFrame(columns=categs, index=df_reduced_test.session_id.values)
catDF.fillna(0, inplace=True)
catDF = catDF.reset_index()
catDF.rename(columns = {'index':'session_id'}, inplace=True)

for i in tqdm(range(len(cat_col_te))):
    try:
        catDF.loc[i, cat_col_te[i]] = 1
    except:
        continue
        
# # train
# categs = set(list(set([x for y in cat_col_te for x in y if x not in ['']])) + list(set([x for y in cat_col_tr for x in y if x not in ['']]))) 
# catDF = pd.DataFrame(columns=categs, index=df_reduced_train.session_id.values)
# catDF.fillna(0, inplace=True)
# catDF = catDF.reset_index()
# catDF.rename(columns = {'index':'session_id'}, inplace=True)

# for i in tqdm(range(len(cat_col_tr))):
#     try:
#         catDF.loc[i, cat_col_tr[i]] = 1
#     except:
#         continue

In [ ]:
df_reduced_test = pd.merge(df_reduced_test, catDF, on="session_id")
df_reduced_test.head(2)

In [ ]:
res = calculate_durations("./data/durations.pkl")
df = pd.merge(df_reduced_test, res, on="session_id", how="outer") # join domain durations

In [ ]:
# replace nans with zeroes, OHE categoricals, drop non-relevant features for training
df.fillna(0, inplace=True)
df = pd.get_dummies(df, columns=["day_start", "day_end", "device_session"])
df.drop(["all_domain", "all_app", "all_category", "start", "end",
         "startend", "session_id"], axis=1, inplace=True)
df.head(2)

In [ ]:
pd.to_pickle(df, "X_test_.pkl")